In [1]:
%load_ext sql
import json
import os
import socket
import cx_Oracle
if os.name == 'nt':
    cx_Oracle.init_oracle_client("C:\\app\\instantclient_21_11")

In [2]:
%%html
<style>
td {text-align: left !important; valign: left !important;}
</style>

# Database migration with the Liberatii Data Platform

The Liberatii Data Platform virtualizes a PostgreSQL database to enable Oracle applications to be migrated to PostgreSQL *without modification*. This notebook contains a tutorial and demonstration of the deployment and use of the Liberatii Data Platform to migrate and test an example Oracle database.

## Architecture

The Liberatii Data Platform uses the Liberatii Gateway to virtualize Azure Databases. This allows
applications built for Oracle to use these databases with **no code changes** and
**no new tests** required.

<img src="Architecture.png" style="margin:auto" alt="Diagram of Liberatii Platform Architecture" title="Architecture"/>

### Change-Data-Capture and Parallel Test Execution

Production workloads can be tested against up-to-date Azure replicas of the Oracle Database
virtualised through Liberatii Gateway using the Change-Data-Capture (CDC) and Parallel Test Execution
features of the Liberatii Data Platform.

<img src="Replay.png" style="margin:auto" alt="Diagram of Workload Replay" title="Replay"/>

This technique provides full confidence for a successful migration **without additional tests**
before any applications are re-targeted to the Liberatii Gateway.

## Demonstration

This notebook uses the
[Oracle H.R. schema](https://docs.oracle.com/cd/E11882_01/server.112/e40540/tablecls.htm#CNCPT111)
to demonstrate the migration of the data and execution of existing Oracle PL/SQL code via
the Liberatii Gateway on a PostgreSQL database.

The H.R. schema provides a basis for queries that use several Oracle-specific features that
are virtualized through the Liberatii Gateway:

<img src="Schema.gif" style="margin:auto" alt="Diagram of the HR Schema" title="Schema"/>

## Migrating an application

This demonstration will migrate the schema and correspdoning SQL with Liberatii Data Platform though the following steps:

1. **Deployment**<br/>
   The deployment of an Azure Managed Application to provide the Liberatii Data Platform
   
3. **Migration**<br/>
   The migration of the schema and data from the Oracle database to PostgreSQL via the Liberatii Gateway
   
5. **Synchronisation**<br/>
   Setup of a Change Data Capture pipeline to synchronize the Oracle and PostgreSQL databases
   
7. **Replay testing**<br/>
   Testing of an Oracle Workload Replay for **performance** and **correctness** against both the Oracle and PostgreSQL databases to verify proper
   operation
   
9. **Switch over**<br/>
   Replacing the database driver and retargeting the application to use the Liberatii Gateway

# Set up

The following cell contains the connection information for the Oracle (Source) and PostgreSQL (Target) databases. It is initially set up to migrate the Oracle demo H.R. schema and expects the given users to exist with the required permissions on the specified databases.

In [3]:
## Hostnames of the databases
PG_HOST='demo-g3rqzbqudztzk-pgsql.postgres.database.azure.com'
ORACLE_HOST='192.168.1.6'
LGW_HOST='192.168.1.4'

## Connection data
DB='pdborcl'
USER='HR'
PSWD='hr'
ORACLE_PORT=1521
PG_PORT=5432
LGW_PORT=5432
PLATFORM=LGW_HOST

## Connection strings (derived from the above connection data)
ORACLE_CONN_STR=f'(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST={ORACLE_HOST})(PORT={ORACLE_PORT}))(CONNECT_DATA=(SERVICE_NAME={DB})))'
PG=f'postgresql://{USER}:{PSWD}@{PG_HOST}:{PG_PORT}/{DB}'
ORACLE=f'oracle://{USER}:{PSWD}@{ORACLE_CONN_STR}'
LGW=f'postgresql://{USER}:{PSWD}@{LGW_HOST}:{LGW_PORT}/{DB}'
API_PREFIX=f"http://{PLATFORM}:3000"

print(f"""
Connection settings:

    PostgreSQL:             
      {PG}
    Oracle:                 
      {ORACLE}
    Liberatii Data Platform:
      {API_PREFIX}/api
    LGW:
      {LGW}
""")


Connection settings:

    PostgreSQL:             
      postgresql://HR:hr@postgres:5432/pdborcl
    Oracle:                 
      oracle://HR:hr@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=oracle)(PORT=1521))(CONNECT_DATA=(SERVICE_NAME=pdborcl)))
    Liberatii Data Platform:
      http://192.168.1.4:3000/api
    LGW:
      postgresql://HR:hr@pgtranslator:15432/pdborcl



In [4]:
%%sql {ORACLE}
-- Find version information for the Oracle database
select banner FROM v$version

0 rows affected.


banner
Oracle Database 19c Enterprise Edition Release 19.0.0.0.0 - Production


In [5]:
%%sql {PG}
-- Find version information for the PostgreSQL database
select version()

1 rows affected.


version
"PostgreSQL 14.9 (Debian 14.9-1.pgdg110+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 10.2.1-6) 10.2.1 20210110, 64-bit"


On LGW we run Oracle SQL query, but it returns PostgreSQL version:

## Test

With the Liberatii Gateway deployed (or otherwise available) we can now test the connection.

The following command uses Oracle SQL with will be translated by the virtualisation layer to access the PostgreSQL database.

In [9]:
%%sql {LGW}
-- Find version information for the PostgreSQL database via Liberatii Gateway
select banner FROM v$version

1 rows affected.


BANNER
"PostgreSQL 14.9 (Debian 14.9-1.pgdg110+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 10.2.1-6) 10.2.1 20210110, 64-bit"


# Migration

The following section shows the migration of the schema and data from the Oracle database to PostgreSQL. This uses the Liberatii Data Platform to perform the following steps:

1. Configuration, connection setup and initialisation
3. Schema Migration
4. Data Migration
5. Testing and verification

All of these steps **can be performed in parallel** across multiple threads and connections to cope with large volumes of data.

## Configuration

The following cells will construct the required connection information. Each connection (Oracle, PostgreSQL and the Liberatii Gateway) will be created in turn.

### Oracle

In [10]:
!curl -s {API_PREFIX}/connection -H "Content-Type: application/json" \
   -d "{{\"type\":\"Oracle\",\"connectionString\":\"{ORACLE_CONN_STR}\",\"user\":\"{USER}\",\"password\":\"{PSWD}\",\"id\":1}}"

{"type":"Oracle","connectionString":"(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=oracle)(PORT=1521))(CONNECT_DATA=(SERVICE_NAME=pdborcl)))","user":"HR","password":"hr","id":1}

### PostgreSQL

In [11]:
!curl -s {API_PREFIX}/connection -H "Content-Type: application/json" \
  -d "{{\"type\":\"PostgreSQL\",\"host\":\"{PG_HOST}\",\"port\":5432,\"database\":\"{DB}\",\"user\":\"{USER}\",\"password\":\"{PSWD}\",\"id\":2}}"

{"type":"PostgreSQL","host":"postgres","port":5432,"database":"pdborcl","user":"HR","password":"hr","id":2}

### Liberatii Gateway:

In [12]:
!curl -s {API_PREFIX}/connection -H "Content-Type: application/json" \
  -d "{{\"type\":\"LGW\",\"host\":\"{LGW_HOST}\",\"port\":{LGW_PORT},\"database\":\"{DB}\",\"user\":\"{USER}\",\"password\":\"{PSWD}\",\"id\":3}}"

{"type":"LGW","host":"pgtranslator","port":15432,"database":"pdborcl","user":"HR","password":"hr","id":3}

### Configuration Parameters

The following configuration parameters are used for this demonstration. Please check the reference document to get all the available parameters.

* `dataOnePass`<br/>
  Don't use staging files or tables, just transfer all the data from the source to the target
* `user`<br/>
  List of schemas to transfer between the sources and target target
* `verbose`<br/>
  Log verbosity
* `eraseOnInit`<br/>
  Delete everything from the schema in the init stage, this is useful if we often restart the migration.

In [13]:
result = !curl -s {API_PREFIX}/config -H "Content-Type: application/json" \
  -d "{{\"dataOnePass\": true, \"users\":[\"{USER}\"], \"verbose\":4, \"eraseOnInit\":true}}"
print(json.dumps(json.loads(result[0]), indent=True))

{
 "message": "Config has been set successfully",
 "config": {
  "dataOnePass": true,
  "verbose": 4,
  "useCopy": true,
  "dataIterations": -1,
  "useWrapper": true,
  "useNative": false,
  "useUnlogged": false,
  "stat": true,
  "statDB": false,
  "rowsBuf": 1000,
  "lightCheck": false,
  "dataChunkSize": -1,
  "bigTablesFirst": true,
  "debReverseOrder": false,
  "cli": true,
  "rmStagingFiles": true,
  "parTables": true,
  "hashType": "murmur",
  "simulateUnsupportedTypes": true,
  "blobStreams": true,
  "clobStreams": false,
  "blobImmed": false,
  "clobImmed": false,
  "dumpCopy": false,
  "dryRun": false,
  "noBlobs": false,
  "idCol": "rowid",
  "idColByTable": {},
  "activeSqlFiles": [],
  "numTries": 10,
  "maxWait": 60000,
  "ignoreTrim": true,
  "removeLastFetchFirst": false,
  "checkMetaData": true,
  "users": [
   "HR"
  ],
  "linkedServers": [],
  "stages": {},
  "workloadsFiles": "",
  "eraseOnInit": true,
  "debezium": "http://kafka-connect:8083/",
  "replayDir": "/tmp

### Initialisation

This is an initialisation operation: the framework runs simple assessments and stores the results in the target database. All operations are **asynchronous** and multiple can be run in **parallel**; this call constructs a new background operation with the specified id that can be used to query its progress.

In [14]:
result = !curl -s {API_PREFIX}/operation -H 'Content-Type: application/json' -d '{{ "id": 1, "oracle": 1, "lgw": 3, "stage": "init" }}'
print(json.dumps(json.loads(result[0]), indent=True))

{
 "message": "Config has been set successfully.",
 "operation": {
  "id": 1,
  "messages": [],
  "progress": 0,
  "status": "Running",
  "title": "",
  "promise": {}
 }
}


Using this operation we can get the current state of the a currently running operation:

In [15]:
result = !curl -s {API_PREFIX}/operation/1?count=10\&pager=10 -H 'Content-Type: application/json'
print(json.dumps(json.loads(result[0]), indent=True))

{
 "id": 1,
 "messages": [],
 "progress": 0,
 "status": "Running",
 "title": "",
 "promise": {}
}


And the next operation is the same as the previous one, except it waits until the current operation ends.

In [16]:
result = !curl -s -X POST {API_PREFIX}/operation/1/wait?pager=10\&count=10 -H 'Content-Type: application/json'
print(json.dumps(json.loads(result[0]), indent=True))

{
 "id": 1,
 "messages": [],
 "progress": 0,
 "status": "Succeeded",
 "title": "",
 "promise": {}
}


## Schema migration

After `init` stage is complete we have all the necessary information for migrating the database schema.

For this demonstration the migration metadata is stored in the PostgreSQL database allowing us to run various queries on it. For example, we can query the number of objects of each type by running with the query:


In [17]:
%%sql {PG}
select count(*), type, stage, error from dbt.migration_objects group by type, stage, error

5 rows affected.


count,type,stage,error
3,SEQUENCE,I,None
11,INDEX,I,None
1,VIEW,I,None
2,PROCEDURE,I,None
7,TABLE,I,None


If there were any errors during the execution of any stage, they will be displayed in `error` stage. The errors can be manually fixed by modifying `ddl1`, `ddl2` columns of `dbt.migration_objects` or by adding some runtime objects:

In [18]:
%%sql {PG}
select ddl1, ddl2 from dbt.migration_objects where type = 'TABLE' limit 1

1 rows affected.


ddl1,ddl2
"CREATE TABLE ""HR"".""COUNTRIES"" ( ""COUNTRY_ID"" CHAR(2) CONSTRAINT ""COUNTRY_ID_NN"" NOT NULL ENABLE, ""COUNTRY_NAME"" VARCHAR2(40), ""REGION_ID"" NUMBER, CONSTRAINT ""COUNTRY_C_ID_PK"" PRIMARY KEY (""COUNTRY_ID"") ENABLE ) ORGANIZATION INDEX NOCOMPRESS","ALTER TABLE ""HR"".""COUNTRIES"" ADD SUPPLEMENTAL LOG DATA (ALL) COLUMNS;ALTER TABLE ""HR"".""COUNTRIES"" ADD CONSTRAINT ""COUNTR_REG_FK"" FOREIGN KEY (""REGION_ID"") REFERENCES ""HR"".""REGIONS"" (""REGION_ID"") ENABLE;"


When errors are fixed we don't need to run the whole migration from the beginning. It's enough to reset only the problematic objects' `stage` field and restart the stage. Only those objects migration will be re-run.

The schema migration doesn't migrate constraints, indexes and triggers. They are migrated in `constraints` stage. This is done to make data migration run faster. The Liberatii Data Platform can also execute **multiple parallel instances** of each operation to signficantly improve transfer speeds. It is even possible to perform **parallel migrations for single tables** by splitting them.

The schema migration is started with same parameters as with `init` stage, but with `schema` as the stage name.

In [19]:
!curl -s {API_PREFIX}/operation -H 'Content-Type: application/json' -d '{{ "id": 1, "oracle": 1, "lgw": 3, "stage": "schema" }}'

{"message":"Config has been set successfully.","operation":{"id":1,"messages":[],"progress":0,"status":"Running","title":"","promise":{}}}

Again, waiting until this task is finished:


In [20]:
!curl -s -X POST {API_PREFIX}/operation/1/wait?count=1 -H 'Content-Type: application/json'

{"id":1,"messages":["2 config: {"],"progress":0,"status":"Succeeded","title":"","promise":{}}

In the `init` stage the framework also queries the sizes of each table. Using them we can get the most optimal strategy for copying the data.

## Data

So let's see the sizes:


In [21]:
%%sql {PG}
select name, pg_size_pretty(data_size), stage, error from dbt.migration_objects where type = 'TABLE' order by data_size desc

7 rows affected.


name,pg_size_pretty,stage,error
REGIONS,64 kB,D,None
EMPLOYEES,64 kB,D,None
LOCATIONS,64 kB,D,None
JOBS,64 kB,D,None
DEPARTMENTS,64 kB,D,None
JOB_HISTORY,64 kB,D,None
COUNTRIES,0 bytes,D,None



By analysing the sizes for different tables we can apply different strategies for different tables. But size the sizes are quire small for this schema we just use the default strategy everywhere.

Now starting the data migration operation, as all the operations before, but with `data` as `stage` field value:


In [22]:
!curl -s {API_PREFIX}/operation -H 'Content-Type: application/json' -d '{{ "id": 1, "oracle": 1, "lgw": 3, "stage": "data" }}'

{"message":"Config has been set successfully.","operation":{"id":1,"messages":[],"progress":0,"status":"Running","title":"","promise":{}}}

Awaiting the operation to be completed:

In [23]:
!curl -s -X POST {API_PREFIX}/operation/1/wait?count=1 -H 'Content-Type: application/json'

{"id":1,"messages":["2 config: {"],"progress":0,"status":"Succeeded","title":"","promise":{}}

## Constraints
This stage adds constraints, indexes and triggers to the data we've moved in the previous step:


In [24]:
!curl -s {API_PREFIX}/operation -H 'Content-Type: application/json' -d '{{ "id": 1, "oracle": 1, "lgw": 3, "stage": "constraints" }}'

{"message":"Config has been set successfully.","operation":{"id":1,"messages":[],"progress":0,"status":"Running","title":"","promise":{}}}

Awaiting the operation to be completed:

In [25]:
!curl -s -X POST {API_PREFIX}/operation/1/wait?count=1 -H 'Content-Type: application/json'

{"id":1,"messages":["2 config: {"],"progress":0,"status":"Succeeded","title":"","promise":{}}

## Verification

With the data in place we can verify the migration across both databases to ensure they are the same. This is done by comparing hash sums run across data in each column:

In [26]:
!curl -s {API_PREFIX}/operation -H 'Content-Type: application/json' -d '{{ "id": 1, "oracle": 1, "lgw": 3, "stage": "check" }}'

{"message":"Config has been set successfully.","operation":{"id":1,"messages":[],"progress":0,"status":"Running","title":"","promise":{}}}

In [27]:
!curl -s -X POST {API_PREFIX}/operation/1/wait?count=1 -H 'Content-Type: application/json'

{"id":1,"messages":["2 config: {"],"progress":0,"status":"Succeeded","title":"","promise":{}}

In [28]:
%%sql {PG}

select stage, type from dbt.migration_objects group by stage, type

5 rows affected.


stage,type
D,SEQUENCE
D,INDEX
D,VIEW
K,TABLE
D,PROCEDURE


This is it, now the whole database is migrated. Please see the guide document for the information about different migration options.

## Running queries

Let's now run a few queries in the both databases to see the results are identical. There is a `checksql` stage to do this automatically, but for the demo purposes we do this manually.

So first we check the databases are indeed Oracle and PostgreSQL:

In [29]:
%%sql {ORACLE}
select banner from v$version

0 rows affected.


banner
Oracle Database 19c Enterprise Edition Release 19.0.0.0.0 - Production


In [30]:
%%sql {LGW}
select banner from v$version

1 rows affected.


BANNER
"PostgreSQL 14.9 (Debian 14.9-1.pgdg110+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 10.2.1-6) 10.2.1 20210110, 64-bit"


The output value on PostgreSQL can be configured to output exactly the same like Oracle if the application uses the data for some business logic selection.

Now let's run the same query in both databases. This query contains a number of Oracle-specific features:

* Concatenation using the double-pipe (||) operator
* The `NVL` and `DECODE` functions
* The join-plus operator and implicit joins

In [31]:
QUERY="""
SELECT first_name  || ' ' ||  last_name full_name,
        salary +
        NVL  (commission_pct, 0) sal_com,
        DECODE (NVL(e.department_id, -3),
                    60, d.department_name,
                    50, d.department_name,
                    30, d.department_name,
                    -3, 'UNKNOWN',
               'OTHER') dep,
        TO_CHAR(e.hire_date) hire_date
FROM employees e, departments d
WHERE d.department_id(+) = e.department_id
        AND e.last_name like 'G%'
ORDER BY first_name  ||  ' '  ||  last_name
"""

First running it on Oracle:

In [32]:
%%sql {ORACLE} 

{QUERY}

0 rows affected.


full_name,sal_com,dep,hire_date
Danielle Greene,9500.15,OTHER,19-MAR-07
Douglas Grant,2600,Shipping,13-JAN-08
Girard Geoni,2800,Shipping,03-FEB-08
Ki Gee,2400,Shipping,12-DEC-07
Kimberely Grant,7000.15,UNKNOWN,24-MAY-07
Nancy Greenberg,12008,OTHER,17-AUG-02
Timothy Gates,2900,Shipping,11-JUL-06
William Gietz,8300,OTHER,07-JUN-02


And now running absolutely the same query but on PostgreSQL via Liberatii Gateway:

In [33]:
%%sql {LGW}

{QUERY}

8 rows affected.


FULL_NAME,SAL_COM,DEP,HIRE_DATE
Danielle Greene,9500.15,OTHER,19-MAR-07
Douglas Grant,2600.00,Shipping,13-JAN-08
Girard Geoni,2800.00,Shipping,03-FEB-08
Ki Gee,2400.00,Shipping,12-DEC-07
Kimberely Grant,7000.15,UNKNOWN,24-MAY-07
Nancy Greenberg,12008.00,OTHER,17-AUG-02
Timothy Gates,2900.00,Shipping,11-JUL-06
William Gietz,8300.00,OTHER,07-JUN-02


The results are identical. The next steps are:

* Synchronise the databases using CDC so we avoid downtimes on switchover
* Replay workloads on a sandbox Oracle database and PostgreSQL via Liberatii Gateway, for testing correctness and performance